# explanation #

## necessity ##
confirm that you can run a fortran compiler, e.g., "gfortran",  and "f2py". 
"f2py" is a part of scipy.

## make libraries ##

change a fortran compiler if necessary and 
execute
```
$ make
```
and confirm that 
```
  myext.so
  mylib.so
  mymod.so
```
are in the directory. Probably you see so many messages and may think that it failed. If you find +x permission, 
```
-rwxrwxr-x 1 kino kino 146800  3月 16 16:37 myext.so
-rwxrwxr-x 1 kino kino 118688  3月 16 16:37 mylib.so
-rwxrwxr-x 1 kino kino 120808  3月 16 16:37 mymod.so
```
it finished successfully. 

## jupyter notebook ##
Then you can run jupyter notebook

## errors in jupyter ##
If an error occurs in running ipynb, then shutdown the process and restart it. 


# Notes #
## Fortran derived types ## 
f2py **does not** natively (as of Jan 2013) directly support Fortran derived types. However, you can trick it to treat them as opaque blobs, as shown here, if you tell it how many bytes it should reserve. The exact number of bytes required may vary with compiler/compiler options, so it should be chosen big enough.

ref.
http://central.scipy.org/item/60/1/f2py-with-opaque-fortran-derived-types

## Other comments ##
- f2py can compile many many files. 
- You can't write derived types anywhere in the fortran code even if it isn't called directly from the interface code that has the !f2py directive. 
- You can use module in the fortan side to save/communicate among fortran routines.
- f2py is designed so that the fortran is only a sub part of the python, which accelarates a function